# 패키지, 파일 불러오기

In [109]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import collections

In [110]:
df = pd.read_csv('tracks_final.csv')

In [111]:
with open("가사.txt", "r", encoding="utf-8") as f:
    lyrics = f.read()


# koNLPy로 가사 단어 tokenize

In [112]:
okt = Okt()

k_nouns = okt.nouns(lyrics)
k_top_nouns = collections.Counter(k_nouns)
# k_top_nouns.most_common()

In [113]:
morph = okt.morphs(lyrics)
pumsa = okt.pos(lyrics)

In [114]:
type(pumsa)

list

In [115]:
type(pumsa[0])

tuple

In [116]:
pumsa[0]

('우리', 'Noun')

In [117]:
pumsa[0][0]

'우리'

In [118]:
# 품사 종류 파악
gra = []
for pum in pumsa:
    gra.append(pum[1])

In [119]:
set(gra)

{'Adjective',
 'Adverb',
 'Alpha',
 'Conjunction',
 'Determiner',
 'Eomi',
 'Exclamation',
 'Foreign',
 'Hashtag',
 'Josa',
 'KoreanParticle',
 'Modifier',
 'Noun',
 'Number',
 'Punctuation',
 'Suffix',
 'URL',
 'Verb',
 'VerbPrefix'}

# 함수 모음

In [120]:
# 가사 텍스트에서 pos 작업을 거쳐 품사별 단어 리스트 만드는 함수
def find_pumsa(text, grammar):
    okt = Okt()
    tagged = okt.pos(text)

    gram_list = []
    
    for tag in tagged:
        if tag[1] == grammar:
            gram_list.append(tag[0])
    
    return gram_list


In [121]:
# pos 작업 완료된 텍스트로 단어별 빈도수 담은 Counter 객체 리턴
# ('단어', 빈도수(숫자)) 모음이 리턴됨
def find_pumsa2(tagged, grammar):

    gram_list = []
    for tag in tagged:
        if tag[1] == grammar:
            gram_list.append(tag[0])

    top_gram = collections.Counter(gram_list)
    top_gram = top_gram.most_common()
    
    return top_gram

In [122]:
# 품사별 Counter 객체로 원하는 단어의 빈도수 뽑는 함수
def count_words(target, words):
    for word in words:
        if word[0] == target:
            return word[1]

In [123]:
# 품사별 Counter 객체로 원하는 단어의 빈도수, 빈도수 순위, 빈도수 순위의 상위 n% 비율 뽑는 함수
def count_and_rank(target, words):
    for i in range(len(words)):
        if words[i][0] == target:
            # return i+1, "등", words[i][1], "개"
            return f"{target}: {i+1}등, {words[i][1]}회, 상위 {round(((i+1) / len(words))*100, 2)}%"

In [124]:
# 리스트 단위로 단어를 삽입하는 함수
def list_count_and_rank(word_list, words):
    for word in word_list:
        print(count_and_rank(word, words)) 

In [125]:
# 원하는 곡의 가사 텍스트를 pos 분석 해주는 함수
def target_song_text(title):
    okt = Okt()
    song = df[(df['제목'] == title)]
    song_txt = ' '.join(song['가사'])
    song_txt = song_txt.replace("\r\n", " ")
    song_pos = okt.pos(song_txt)
    return song_pos

# 품사별 단어 등장횟수 뽑기

## 품사별 pos 적용 (find_pumsa2)

In [126]:
k_nouns = find_pumsa2(pumsa, 'Noun')
k_adj = find_pumsa2(pumsa, 'Adjective')
k_adv = find_pumsa2(pumsa, 'Adverb')
k_alpha = find_pumsa2(pumsa, 'Alpha')
k_conj = find_pumsa2(pumsa, 'Conjunction')
k_verb = find_pumsa2(pumsa, 'Verb')
k_excla = find_pumsa2(pumsa, 'Exclamation')
k_for = find_pumsa2(pumsa, 'Foreign')

## 내가 찾고 싶은 단어

In [127]:
# 술, 담배, 커피
words_list = ['술', '담배', '커피']
for word in words_list:
    print(count_and_rank(word, k_nouns))

술: 62등, 70회, 상위 1.32%
담배: 201등, 24회, 상위 4.28%
커피: 450등, 11회, 상위 9.59%


In [128]:
print(count_and_rank('coffee', k_alpha))
print(count_and_rank('Coffee', k_alpha))

coffee: 3164등, 1회, 상위 98.91%
Coffee: 536등, 7회, 상위 16.76%


In [129]:
# 욕설
mine1 = ['fuck', 'Fuck', 'fucking', "fuckin'", "fxxkin'", 'fxxk', 'Fxxk']
list_count_and_rank(mine1, k_alpha)

fuck: 98등, 41회, 상위 3.06%
Fuck: 639등, 5회, 상위 19.97%
None
None
None
None
None


In [130]:
# 돈
print(count_and_rank('돈', k_nouns))

mine2 = ['Money', 'money', 'MONEY']
list_count_and_rank(mine2, k_alpha)

돈: 63등, 70회, 상위 1.34%
None
money: 440등, 9회, 상위 13.75%
None


In [131]:
# 사람
mine3 = ['사람', '남자', '여자']
mine4 = ['human', 'Human', 'man', 'Man', 'men', 'Men', 'woman', 'Woman', 'women', 'Women']
list_count_and_rank(mine3, k_nouns)
list_count_and_rank(mine4, k_alpha)

사람: 48등, 86회, 상위 1.02%
남자: 125등, 38회, 상위 2.66%
여자: 301등, 16회, 상위 6.41%
human: 484등, 8회, 상위 15.13%
None
man: 102등, 39회, 상위 3.19%
Man: 392등, 10회, 상위 12.25%
men: 2513등, 1회, 상위 78.56%
None
None
None
None
None


In [132]:
for i in pumsa:
    if i[0] == '외롭':
        print(i)

In [133]:
# 춥다, 외롭다
print(count_and_rank('춥다', k_nouns))
print(count_and_rank('외로움', k_nouns))

춥다: 337등, 14회, 상위 7.18%
외로움: 419등, 11회, 상위 8.93%


In [134]:
count_and_rank('구원', k_nouns)

'구원: 182등, 28회, 상위 3.88%'

In [135]:
count_and_rank('이별', k_nouns)

'이별: 128등, 36회, 상위 2.73%'

In [136]:
count_and_rank('눈', k_nouns)

'눈: 14등, 193회, 상위 0.3%'

## 신청 받은 단어

In [137]:
# 사랑
print(count_and_rank('love', k_alpha))
print(count_and_rank('Love', k_alpha))
print(count_and_rank("사랑", k_nouns))

love: 17등, 179회, 상위 0.53%
Love: 38등, 108회, 상위 1.19%
사랑: 7등, 289회, 상위 0.15%


In [138]:
# 번뇌, 독백, 고뇌, 타인
req1 = ['번뇌', '독백', '고뇌', '타인']
list_count_and_rank(req1, k_nouns)

번뇌: 4064등, 1회, 상위 86.58%
독백: 1578등, 3회, 상위 33.62%
고뇌: 2068등, 2회, 상위 44.06%
타인: 657등, 7회, 상위 14.0%


In [139]:
# 너, 넌, 그대, 사람
req1_1 = ['너', '넌', '그대', '사람']
list_count_and_rank(req1_1, k_nouns)

너: 3등, 473회, 상위 0.06%
넌: 18등, 167회, 상위 0.38%
그대: 20등, 152회, 상위 0.43%
사람: 48등, 86회, 상위 1.02%


In [140]:
# 우리
count_and_rank('우리', k_nouns)

'우리: 36등, 102회, 상위 0.77%'

In [141]:
# 그대, 밤, 죄, 숨, 손, 심장, 도시, 세상
req2 = ['그대', '밤', '죄', '숨', '손', '심장', '도시', '세상']
list_count_and_rank(req2, k_nouns)

그대: 20등, 152회, 상위 0.43%
밤: 26등, 128회, 상위 0.55%
죄: 146등, 33회, 상위 3.11%
숨: 29등, 117회, 상위 0.62%
손: 15등, 179회, 상위 0.32%
심장: 72등, 62회, 상위 1.53%
도시: 80등, 56회, 상위 1.7%
세상: 10등, 256회, 상위 0.21%


In [142]:
# 당신
count_and_rank('당신', k_nouns)

'당신: 74등, 60회, 상위 1.58%'

In [143]:
# epik, high
req3 = ['Epik', 'epik', 'EPIK', 'Epic', 'epic', 'EPIC', 'High', 'high', 'HIGH']
req3_1 = ['에픽', '하이', '에픽하이']
list_count_and_rank(req3, k_alpha)
print("-------")
list_count_and_rank(req3_1, k_nouns)

Epik: 92등, 44회, 상위 2.88%
epik: 283등, 16회, 상위 8.85%
EPIK: 657등, 5회, 상위 20.54%
None
epic: 1600등, 2회, 상위 50.02%
None
High: 85등, 48회, 상위 2.66%
high: 32등, 127회, 상위 1.0%
HIGH: 2314등, 1회, 상위 72.34%
-------
에픽: 319등, 15회, 상위 6.8%
None
에픽하이: 591등, 8회, 상위 12.59%


In [144]:
# 죽음 관련 단어
print(count_and_rank('죽음', k_nouns))
print(count_and_rank('죽', k_nouns))
print(count_and_rank('죽', k_verb))
print(count_and_rank('죽다', k_verb))
print(count_and_rank('죽', k_adv))
print(count_and_rank('죽', k_adj))

죽음: 294등, 17회, 상위 6.26%
죽: 423등, 11회, 상위 9.01%
None
죽다: 2403등, 1회, 상위 63.17%
None
None


In [145]:
# Soul
req4 = ['Soul', 'soul', 'SOUL']
list_count_and_rank(req4, k_alpha)

Soul: 569등, 6회, 상위 17.79%
soul: 95등, 44회, 상위 2.97%
None


In [146]:
count_and_rank("Motherfucker", k_alpha)

'Motherfucker: 1622등, 1회, 상위 50.7%'

In [147]:
count_and_rank("날개", k_nouns)

'날개: 67등, 66회, 상위 1.43%'

In [148]:
# 꿈, Love, 운명, 포기
req5_1 = ['꿈', '운명', '포기']
req5_2 = ['love', 'Love', 'destiny', 'Destiny', 'dream', 'Dream']
list_count_and_rank(req5_1, k_nouns)
print("--------")
list_count_and_rank(req5_2, k_alpha)

꿈: 16등, 170회, 상위 0.34%
운명: 177등, 28회, 상위 3.77%
포기: 240등, 20회, 상위 5.11%
--------
love: 17등, 179회, 상위 0.53%
Love: 38등, 108회, 상위 1.19%
destiny: 1202등, 2회, 상위 37.57%
Destiny: 112등, 36회, 상위 3.5%
dream: 461등, 8회, 상위 14.41%
Dream: 1896등, 1회, 상위 59.27%


## 원하는 곡별 텍스트 분석

In [149]:
love_song = target_song_text("Love Love Love (Feat. Yoong Jin of Casker))")

# 품사별 단어 경향성 분석

In [150]:
# 1회만 등장하는 단어의 갯수와 비중은?
nouns = []
print(f"명사 전체: {len(k_nouns)}개")

for noun in k_nouns:
    if noun[1] == 1:
        nouns.append(noun[0])
print(f"1회만 등장: {len(nouns)}개, 전체 중 {round((len(nouns) / len(k_nouns))*100, 2)}%")

명사 전체: 4694개
1회만 등장: 2345개, 전체 중 49.96%


In [151]:
k_nouns[0]

('내', 982)

In [152]:
# 수가 같은 경우는 어떻게 하지...?
# 등장횟수 같은 것들끼리 전부 묶어보기
counts = []
for noun in k_nouns:
    counts.append(noun[1])

In [153]:
# 상위 10% 컷은 몇 차례 등장, 상위 5% 컷은 몇 차례 등장인지 같은 거 뽑아보기
for i in range(len(k_nouns)):
    if (i / len(k_nouns))*100 > 10:
        print(f"상위 10% 컷: {k_nouns[i][1]}회 등장까지")
        break

상위 10% 컷: 10회 등장까지


In [154]:
for i in range(len(k_nouns)):
    if (i / len(k_nouns))*100 > 5:
        print(f"상위 5% 컷: {k_nouns[i][1]}회 등장까지")
        break

상위 5% 컷: 21회 등장까지


# TMI

## 19금

In [155]:
pa = []
for i in df['제목']:
    if i[:3] == '19금':
        pa.append(i)

In [156]:
df.loc[1]['제목']

'그래서 그래 (Feat. 윤하)'

In [157]:
df.loc[1]['제목'][:3]

'그래서'

In [158]:
titles = []

for i in range(len(df)):
    if df.loc[i]['제목'][:3] == '19금':
        titles.append(df.loc[i])


In [159]:
pa[0]

'19금 노땡큐 (Feat. MINO & 사이먼 도미닉 & 더콰이엇)'

In [160]:
len(pa)

23

In [161]:
pa1 = pd.concat(titles)
# print(pa1)

In [162]:
pa1['제목'].unique()

array(['19금 노땡큐 (Feat. MINO & 사이먼 도미닉 & 더콰이엇)',
       '19금 BORN HATER (Feat. 빈지노 & 버벌진트 & B.I & MINO & BOBBY)',
       '19금 Rosario (Feat. CL & ZICO)', '19금 정당방위 (Feat. 우원재 & 넉살 & 창모)',
       '19금 8 By 8, Part 2 (Bonus Track) (Feat. MYK, Minos, Paloalto, The Quiett, Verbal Jint, Kebee, E-Sens, Simon Dominic)',
       '19금 Mr. Doctor', '19금 피해망상 Pt.3 With TBNY', '19금 Flow',
       '19금 흉 (Feat. MYK & YDG & Dok2)', '19금 뒷담화',
       '19금 Rocksteady (Feat. Paloalto & Dok2 & Beatbox DG & Beenzino) (Korean Ver.)',
       '19금 FAQ', '19금 말로맨',
       '19금 Rocksteady (Feat. Kero One & Dumbfoundead & MYK & Rakka (Dilated Peoples))',
       '19금 Supreme 100', '19금 B-side 01: Breakdown (Supreme Mix)',
       '19금 Still Here (Feat. Dok2)',
       '19금 B-Side 01 : Breakdown (Supreme mix)', '19금 Butterfly Effect',
       '19금 신사들의 몰락 (Good Evening)', '19금 신사들의 절약정신 (Good Afternoon)',
       '19금 Eight By Eight (Feat. Dynamic Duo, Dok2, Double K, TBNY)',
       '19금 The Future (Feat. Yankie)'], dt

In [163]:
len(pa)

23